### 参照設定

In [1]:
#r "nuget:Microsoft.Data.Sqlite,8.0.3"
#r "nuget:ccxt,4.2.85"
#r "/workspace/src/Common/obj/Debug/net8.0/Common.dll"
#r "/workspace/src/Domain/obj/Debug/net8.0/Domain.dll"
#r "/workspace/src/Infra/obj/Debug/net8.0/Infra.dll"

using Microsoft.Extensions.Logging;
using BotTrade.Infra;
using BotTrade.Infra.Exchanges;
using BotTrade.Domain;
using BotTrade.Domain.Strategies;


Installed Packages ccxt, 4.2.85 Microsoft.Data.Sqlite, 8.0.3

### 設定

In [2]:
var setting = new ExchangeSetting() 
{ 
    Place = ExchangePlace.Bitfinex, 
    Symbol = Symbol.Spot_BTCUSD
};


### DB更新

In [3]:
var factory = LoggerFactory.Create(builder => 
{
    builder.AddConsole();
});
var logger = factory.CreateLogger<PastCandleRepository>();
var repository = new PastCandleRepository(setting, logger);

await repository.Fetch();


info: BotTrade.Infra.PastCandleRepository[0]
      前回更新時: 1721049480000
info: BotTrade.Infra.PastCandleRepository[0]
      True
info: BotTrade.Infra.PastCandleRepository[0]
      Fetched since: 07/14/2024 20:42:23 +00:00, count: 3
info: BotTrade.Infra.PastCandleRepository[0]
      データベース更新完了


### バックアップ作成

In [4]:
#!set --value @csharp:setting --name setting
$place = $null
switch($setting.RootElement.GetProperty("Place").GetInt64()) {
    0 { $place = "Bybit" }
#   1 { $place = "BybitTestnet" }
    2 { $place = "Binance" }
#   3 { $place = "BinanceTestnet" }
    4 { $place = "Bitfinex" }
    5 { $place = "Bitfinex2"}
    6 { $place = "Bitmex" }
    7 { $place = "Bitbank" }
    default { echo "不明な取引所のためバックアップを行いません" }
}
if($place -eq $null) {
    return $null
}

$data_dir = "/workspace/data"
$db_file = "{0}/{1}.sqlite3" -f $data_dir, $place
$backup_file = "{0}/backup/{1}.backup" -f $data_dir, $place

sqlite3 $db_file .dump > $backup_file
